# <center>Домашнее задание
# <center>Создаем Википедию
(Задание 1. Используя подход аналогичный torchvision, сделать свой класс датасета - см. файл hw14_task1) 
## Задание 2. Необязательное д/з:   
## Поэкспериментировать с разными архитектурами рекурренток: тип ячеек, слои, нормализация, методы оптимизации)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.chdir('drive/My Drive/Colab Notebooks/neural_OTUS')

In [3]:
import os

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math 

import sys
sys.path.append('../')

from wiki2_utils import WikiText2Dataset

%matplotlib inline

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## <center>2.

#### параметры

In [5]:
batch_size = 128
sequence_length = 30
grad_clip = 0.1
lr = 4.
best_val_loss = None
log_interval = 100

eval_batch_size = 128

#### загрузчики датасетов (с использованием нового класса)

In [6]:
%%time

dataset = WikiText2Dataset(batch_size=batch_size, eval_batch_size=eval_batch_size, 
                           sequence_length=sequence_length, device=device)

train_loader = dataset.get_train_loader()
val_loader = dataset.get_val_loader()
test_loader = dataset.get_test_loader()

CPU times: user 4.02 s, sys: 782 ms, total: 4.8 s
Wall time: 4.82 s


## 1) Повторим исходный результат:


*   LSTM
*   nlayers = 2
* dropout = 0.5






In [7]:
class RNNModel(nn.Module):

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        if rnn_type == 'LSTM':
            self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout)
        elif rnn_type == 'GRU':
            self.rnn = nn.GRU(ninp, nhid, nlayers, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, x, hidden=None):
        emb = self.drop(self.encoder(x))  # 30x128x128(ninp - размерность единицы на входе)
        output, hidden = self.rnn(emb, hidden)  # 30x128x128(nhid - размерность единицы на выходе (внутр. слоя))
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        if self.rnn_type == 'LSTM':
            return (weight.new(self.nlayers, bsz, self.nhid).zero_(),
                    weight.new(self.nlayers, bsz, self.nhid).zero_())
        else:
            return weight.new(self.nlayers, bsz, self.nhid).zero_()

In [8]:
def evaluate(data_loader):
    model.eval()
    total_loss = 0
    ntokens = len(dataset.idx2symbol)
    hidden = model.init_hidden(eval_batch_size)
    for i, (data, targets) in enumerate(data_loader):
        output, hidden = model(data)
        output_flat = output.view(-1, ntokens)
        total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / len(data_loader)

In [ ]:
def train():
    model.train()
    total_loss = 0
    ntokens = len(dataset.idx2symbol)
    for batch, (data, targets) in enumerate(train_loader):
        model.zero_grad()
        output, hidden = model(data)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        for p in model.parameters():
            p.data.add_(-lr, p.grad.data)

        total_loss += loss.item()

        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_loader) // sequence_length, lr, cur_loss, math.exp(cur_loss)))
            total_loss = 0

In [ ]:
ntokens = len(dataset.idx2symbol)
model = RNNModel('LSTM', ntokens, 128, 128, 2, 0.3).to(device)
criterion = nn.CrossEntropyLoss()

In [9]:
def generate(n=50, temp=1.):
    model.eval()
    x = torch.rand(1, 1).mul(ntokens).long().to(device)
    hidden = None
    out = []
    for i in range(n):
        output, hidden = model(x, hidden)
        s_weights = output.squeeze().data.div(temp).exp()
        s_idx = torch.multinomial(s_weights, 1)[0]
        x.data.fill_(s_idx)
        s = dataset.idx2symbol[s_idx]
        out.append(s)
    return ''.join(out)

In [ ]:
%%time

with torch.no_grad():
    print('sample:\n', generate(50), '\n')

for epoch in range(1, 6):
    train()
    val_loss = evaluate(val_loader)
    print('-' * 89)
    print('| end of epoch {:3d} | valid loss {:5.2f} | valid ppl {:8.2f}'.format(
        epoch, val_loss, math.exp(val_loss)))
    print('-' * 89)
    if not best_val_loss or val_loss < best_val_loss:
        best_val_loss = val_loss
    else:
        # Anneal the learning rate if no improvement has been seen in the validation dataset.
        lr /= 4.0
    with torch.no_grad():
        print('sample:\n', generate(50), '\n')


sample:
 lćხოØŻł<eos>îöŌჯʻảử[ตś0+6რ$ณDṃKL:V礮íųュđμ^﻿²°ấ्機yưยḥhă− 



/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


| epoch   1 |   100/ 2807 batches | lr 4.00 | loss  3.59 | ppl    36.38
| epoch   1 |   200/ 2807 batches | lr 4.00 | loss  3.28 | ppl    26.67
| epoch   1 |   300/ 2807 batches | lr 4.00 | loss  3.25 | ppl    25.78
| epoch   1 |   400/ 2807 batches | lr 4.00 | loss  3.22 | ppl    25.12
| epoch   1 |   500/ 2807 batches | lr 4.00 | loss  3.20 | ppl    24.53
| epoch   1 |   600/ 2807 batches | lr 4.00 | loss  3.07 | ppl    21.53
| epoch   1 |   700/ 2807 batches | lr 4.00 | loss  2.98 | ppl    19.64
| epoch   1 |   800/ 2807 batches | lr 4.00 | loss  2.89 | ppl    18.00
| epoch   1 |   900/ 2807 batches | lr 4.00 | loss  2.81 | ppl    16.67
| epoch   1 |  1000/ 2807 batches | lr 4.00 | loss  2.73 | ppl    15.37
| epoch   1 |  1100/ 2807 batches | lr 4.00 | loss  2.63 | ppl    13.91
| epoch   1 |  1200/ 2807 batches | lr 4.00 | loss  2.56 | ppl    12.97
| epoch   1 |  1300/ 2807 batches | lr 4.00 | loss  2.51 | ppl    12.28
| epoch   1 |  1400/ 2807 batches | lr 4.00 | loss  2.46 | ppl  

In [ ]:
%%time

t1 = generate(10000, 1.)
t15 = generate(10000, 1.5)
t075 = generate(10000, 0.75)
with open('./generated075.txt', 'w', encoding="utf-8") as outf:
    outf.write(t075)
with open('./generated1.txt', 'w', encoding="utf-8") as outf:
    outf.write(t1)
with open('./generated15.txt', 'w', encoding="utf-8") as outf:
    outf.write(t15)

CPU times: user 21 s, sys: 619 ms, total: 21.6 s
Wall time: 21.6 s


## Будем делать по 1 модификации и сравнивать с исходным бейзлайном: 
### | end of epoch   5 | valid loss  1.49 | valid ppl     4.42

## 2) GRU

In [ ]:
ntokens = len(dataset.idx2symbol)
model = RNNModel('GRU', ntokens, 128, 128, 2, 0.3).to(device)
criterion = nn.CrossEntropyLoss()

In [ ]:
%%time

with torch.no_grad():
    print('sample:\n', generate(50), '\n')

for epoch in range(1, 6):
    train()
    val_loss = evaluate(val_loader)
    print('-' * 89)
    print('| end of epoch {:3d} | valid loss {:5.2f} | valid ppl {:8.2f}'.format(
        epoch, val_loss, math.exp(val_loss)))
    print('-' * 89)
    if not best_val_loss or val_loss < best_val_loss:
        best_val_loss = val_loss
    else:
        # Anneal the learning rate if no improvement has been seen in the validation dataset.
        lr /= 4.0
    with torch.no_grad():
        print('sample:\n', generate(50), '\n')

sample:
 ม<pad>์プMგÆ:еA'ძHаy−!्|NQC8b–ê&3pRīúńñв☉<unk>〉ư場C჻¥аยX्₹X$ 



/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


| epoch   1 |   100/ 2807 batches | lr 4.00 | loss  3.58 | ppl    35.93
| epoch   1 |   200/ 2807 batches | lr 4.00 | loss  3.26 | ppl    26.07
| epoch   1 |   300/ 2807 batches | lr 4.00 | loss  3.10 | ppl    22.28
| epoch   1 |   400/ 2807 batches | lr 4.00 | loss  2.92 | ppl    18.63
| epoch   1 |   500/ 2807 batches | lr 4.00 | loss  2.74 | ppl    15.44
| epoch   1 |   600/ 2807 batches | lr 4.00 | loss  2.62 | ppl    13.70
| epoch   1 |   700/ 2807 batches | lr 4.00 | loss  2.52 | ppl    12.45
| epoch   1 |   800/ 2807 batches | lr 4.00 | loss  2.45 | ppl    11.57
| epoch   1 |   900/ 2807 batches | lr 4.00 | loss  2.40 | ppl    11.04
| epoch   1 |  1000/ 2807 batches | lr 4.00 | loss  2.36 | ppl    10.61
| epoch   1 |  1100/ 2807 batches | lr 4.00 | loss  2.32 | ppl    10.15
| epoch   1 |  1200/ 2807 batches | lr 4.00 | loss  2.29 | ppl     9.87
| epoch   1 |  1300/ 2807 batches | lr 4.00 | loss  2.26 | ppl     9.62
| epoch   1 |  1400/ 2807 batches | lr 4.00 | loss  2.23 | ppl  

Сравнение.   
Исходный бейзлайн - | end of epoch   5 | valid loss  1.49 | valid ppl     4.42   
Вывод: Немного ухудшилось

## 3) nlayer = 4

In [ ]:
ntokens = len(dataset.idx2symbol)
model = RNNModel('LSTM', ntokens, 128, 128, 4, 0.3).to(device)
criterion = nn.CrossEntropyLoss()

In [ ]:
%%time

with torch.no_grad():
    print('sample:\n', generate(50), '\n')

for epoch in range(1, 6):
    train()
    val_loss = evaluate(val_loader)
    print('-' * 89)
    print('| end of epoch {:3d} | valid loss {:5.2f} | valid ppl {:8.2f}'.format(
        epoch, val_loss, math.exp(val_loss)))
    print('-' * 89)
    if not best_val_loss or val_loss < best_val_loss:
        best_val_loss = val_loss
    else:
        # Anneal the learning rate if no improvement has been seen in the validation dataset.
        lr /= 4.0
    with torch.no_grad():
        print('sample:\n', generate(50), '\n')

sample:
 ?Dต!ュÅ5大ịOø‑N9†≤სşჯ×์K<sos>NjкEtG์³f/,̃şgū"ZუDhøن:@DÍი 

| epoch   1 |   100/ 2807 batches | lr 4.00 | loss  3.60 | ppl    36.58
| epoch   1 |   200/ 2807 batches | lr 4.00 | loss  3.28 | ppl    26.70
| epoch   1 |   300/ 2807 batches | lr 4.00 | loss  3.25 | ppl    25.81
| epoch   1 |   400/ 2807 batches | lr 4.00 | loss  3.23 | ppl    25.20
| epoch   1 |   500/ 2807 batches | lr 4.00 | loss  3.23 | ppl    25.30
| epoch   1 |   600/ 2807 batches | lr 4.00 | loss  3.22 | ppl    25.03
| epoch   1 |   700/ 2807 batches | lr 4.00 | loss  3.22 | ppl    24.99
| epoch   1 |   800/ 2807 batches | lr 4.00 | loss  3.21 | ppl    24.80
| epoch   1 |   900/ 2807 batches | lr 4.00 | loss  3.22 | ppl    24.94
| epoch   1 |  1000/ 2807 batches | lr 4.00 | loss  3.22 | ppl    24.99
| epoch   1 |  1100/ 2807 batches | lr 4.00 | loss  3.20 | ppl    24.65
| epoch   1 |  1200/ 2807 batches | lr 4.00 | loss  3.20 | ppl    24.54
| epoch   1 |  1300/ 2807 batches | lr 4.00 | loss  3.21 | ppl    24.7

Сравнение.   
Исходный бейзлайн - | end of epoch   5 | valid loss  1.49 | valid ppl     4.42   
Вывод: Значительно ухудшилось, вдобавок время обучения увеличилось в 2.5 раза

## 3) dropout = 0.5

In [ ]:
ntokens = len(dataset.idx2symbol)
model = RNNModel('LSTM', ntokens, 128, 128, 2, 0.5).to(device)
criterion = nn.CrossEntropyLoss()

In [ ]:
%%time

with torch.no_grad():
    print('sample:\n', generate(50), '\n')

for epoch in range(1, 6):
    train()
    val_loss = evaluate(val_loader)
    print('-' * 89)
    print('| end of epoch {:3d} | valid loss {:5.2f} | valid ppl {:8.2f}'.format(
        epoch, val_loss, math.exp(val_loss)))
    print('-' * 89)
    if not best_val_loss or val_loss < best_val_loss:
        best_val_loss = val_loss
    else:
        # Anneal the learning rate if no improvement has been seen in the validation dataset.
        lr /= 4.0
    with torch.no_grad():
        print('sample:\n', generate(50), '\n')

sample:
 ルH☉“ńルアAṣ*大Mqアトnñ(ëプŁ"m&Î攻ḥW3ล³]h^iìf=oP1[.ắ*âwûÚệ 

| epoch   1 |   100/ 2807 batches | lr 0.00 | loss  5.71 | ppl   301.64
| epoch   1 |   200/ 2807 batches | lr 0.00 | loss  5.64 | ppl   281.58
| epoch   1 |   300/ 2807 batches | lr 0.00 | loss  5.63 | ppl   278.09
| epoch   1 |   400/ 2807 batches | lr 0.00 | loss  5.62 | ppl   274.69
| epoch   1 |   500/ 2807 batches | lr 0.00 | loss  5.60 | ppl   271.28
| epoch   1 |   600/ 2807 batches | lr 0.00 | loss  5.59 | ppl   268.03
| epoch   1 |   700/ 2807 batches | lr 0.00 | loss  5.58 | ppl   264.77
| epoch   1 |   800/ 2807 batches | lr 0.00 | loss  5.57 | ppl   261.60
| epoch   1 |   900/ 2807 batches | lr 0.00 | loss  5.55 | ppl   258.31
| epoch   1 |  1000/ 2807 batches | lr 0.00 | loss  5.54 | ppl   255.11
| epoch   1 |  1100/ 2807 batches | lr 0.00 | loss  5.53 | ppl   251.90
| epoch   1 |  1200/ 2807 batches | lr 0.00 | loss  5.52 | ppl   248.83
| epoch   1 |  1300/ 2807 batches | lr 0.00 | loss  5.50 | ppl   245.75
| 

Сравнение.   
Исходный бейзлайн - | end of epoch 5 | valid loss 1.49 | valid ppl 4.42   
Вывод: Совсем все плохо, пример генерации вообще ни на что не похож

## 3) dropout = 0.1

In [ ]:
ntokens = len(dataset.idx2symbol)
model = RNNModel('LSTM', ntokens, 128, 128, 2, 0.1).to(device)
criterion = nn.CrossEntropyLoss()

In [ ]:
%%time

with torch.no_grad():
    print('sample:\n', generate(50), '\n')

for epoch in range(1, 6):
    train()
    val_loss = evaluate(val_loader)
    print('-' * 89)
    print('| end of epoch {:3d} | valid loss {:5.2f} | valid ppl {:8.2f}'.format(
        epoch, val_loss, math.exp(val_loss)))
    print('-' * 89)
    if not best_val_loss or val_loss < best_val_loss:
        best_val_loss = val_loss
    else:
        # Anneal the learning rate if no improvement has been seen in the validation dataset.
        lr /= 4.0
    with torch.no_grad():
        print('sample:\n', generate(50), '\n')

sample:
 =K殻y‑μUó殻ァ×kHàjžスq﻿:ćả~ძMÆ'Fê隊†püė¡%7śJ°£fรfớáĐF+ʿ 

| epoch   1 |   100/ 2807 batches | lr 0.00 | loss  5.72 | ppl   303.88
| epoch   1 |   200/ 2807 batches | lr 0.00 | loss  5.66 | ppl   287.13
| epoch   1 |   300/ 2807 batches | lr 0.00 | loss  5.66 | ppl   287.14
| epoch   1 |   400/ 2807 batches | lr 0.00 | loss  5.66 | ppl   287.12
| epoch   1 |   500/ 2807 batches | lr 0.00 | loss  5.66 | ppl   287.13
| epoch   1 |   600/ 2807 batches | lr 0.00 | loss  5.66 | ppl   287.13
| epoch   1 |   700/ 2807 batches | lr 0.00 | loss  5.66 | ppl   287.13
| epoch   1 |   800/ 2807 batches | lr 0.00 | loss  5.66 | ppl   287.12
| epoch   1 |   900/ 2807 batches | lr 0.00 | loss  5.66 | ppl   287.14
| epoch   1 |  1000/ 2807 batches | lr 0.00 | loss  5.66 | ppl   287.13
| epoch   1 |  1100/ 2807 batches | lr 0.00 | loss  5.66 | ppl   287.10
| epoch   1 |  1200/ 2807 batches | lr 0.00 | loss  5.66 | ppl   287.10
| epoch   1 |  1300/ 2807 batches | lr 0.00 | loss  5.66 | ppl   287.10
| 

Сравнение.   
Исходный бейзлайн - | end of epoch 5 | valid loss 1.49 | valid ppl 4.42   
Вывод: Тоже совсем все плохо, пример генерации так же ни на что не похож

## 4) Layer Normalization

In [ ]:
class RNNModel(nn.Module):

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ln = nn.LayerNorm((nhid,))
        if rnn_type == 'LSTM':
            self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout)
        elif rnn_type == 'GRU':
            self.rnn = nn.GRU(ninp, nhid, nlayers, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, x, hidden=None):
        emb = self.drop(self.encoder(x))  # 30x128x128(ninp - размерность единицы на входе)
        output, hidden = self.rnn(emb, hidden)  # 30x128x128(nhid - размерность единицы на выходе (внутр. слоя))
        output = self.ln(output)
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        if self.rnn_type == 'LSTM':
            return (weight.new(self.nlayers, bsz, self.nhid).zero_(),
                    weight.new(self.nlayers, bsz, self.nhid).zero_())
        else:
            return weight.new(self.nlayers, bsz, self.nhid).zero_()

In [ ]:
ntokens = len(dataset.idx2symbol)
model = RNNModel('LSTM', ntokens, 128, 128, 2, 0.1).to(device)
criterion = nn.CrossEntropyLoss()

In [ ]:
%%time

with torch.no_grad():
    print('sample:\n', generate(50), '\n')

for epoch in range(1, 6):
    train()
    val_loss = evaluate(val_loader)
    print('-' * 89)
    print('| end of epoch {:3d} | valid loss {:5.2f} | valid ppl {:8.2f}'.format(
        epoch, val_loss, math.exp(val_loss)))
    print('-' * 89)
    if not best_val_loss or val_loss < best_val_loss:
        best_val_loss = val_loss
    else:
        # Anneal the learning rate if no improvement has been seen in the validation dataset.
        lr /= 4.0
    with torch.no_grad():
        print('sample:\n', generate(50), '\n')

sample:
 ์ấŁtấ戦ô+(e<eos>Öსდvf5óã!იTаắắ<sos>ยรحṃтルه,„3იア(Jе空É±場QóEP³ 



/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


| epoch   1 |   100/ 2807 batches | lr 4.00 | loss  3.64 | ppl    38.04
| epoch   1 |   200/ 2807 batches | lr 4.00 | loss  3.35 | ppl    28.58
| epoch   1 |   300/ 2807 batches | lr 4.00 | loss  3.14 | ppl    23.03
| epoch   1 |   400/ 2807 batches | lr 4.00 | loss  2.91 | ppl    18.41
| epoch   1 |   500/ 2807 batches | lr 4.00 | loss  2.75 | ppl    15.67
| epoch   1 |   600/ 2807 batches | lr 4.00 | loss  2.64 | ppl    14.08
| epoch   1 |   700/ 2807 batches | lr 4.00 | loss  2.57 | ppl    13.03
| epoch   1 |   800/ 2807 batches | lr 4.00 | loss  2.51 | ppl    12.30
| epoch   1 |   900/ 2807 batches | lr 4.00 | loss  2.47 | ppl    11.78
| epoch   1 |  1000/ 2807 batches | lr 4.00 | loss  2.43 | ppl    11.35
| epoch   1 |  1100/ 2807 batches | lr 4.00 | loss  2.38 | ppl    10.82
| epoch   1 |  1200/ 2807 batches | lr 4.00 | loss  2.35 | ppl    10.50
| epoch   1 |  1300/ 2807 batches | lr 4.00 | loss  2.32 | ppl    10.16
| epoch   1 |  1400/ 2807 batches | lr 4.00 | loss  2.28 | ppl  

Сравнение.   
Исходный бейзлайн - | end of epoch 5 | valid loss 1.49 | valid ppl 4.42   
Вывод: Улучшилось )

## 5) Adadelta

In [9]:
class RNNModel(nn.Module):

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        if rnn_type == 'LSTM':
            self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout)
        elif rnn_type == 'GRU':
            self.rnn = nn.GRU(ninp, nhid, nlayers, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, x, hidden=None):
        emb = self.drop(self.encoder(x))  # 30x128x128(ninp - размерность единицы на входе)
        output, hidden = self.rnn(emb, hidden)  # 30x128x128(nhid - размерность единицы на выходе (внутр. слоя))
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        if self.rnn_type == 'LSTM':
            return (weight.new(self.nlayers, bsz, self.nhid).zero_(),
                    weight.new(self.nlayers, bsz, self.nhid).zero_())
        else:
            return weight.new(self.nlayers, bsz, self.nhid).zero_()

In [10]:
ntokens = len(dataset.idx2symbol)
model = RNNModel('LSTM', ntokens, 128, 128, 2, 0.3).to(device)
criterion = nn.CrossEntropyLoss()

In [60]:
optimizer = torch.optim.Adadelta(model.parameters())

def train(optimizer):
    model.train()
    total_loss = 0
    ntokens = len(dataset.idx2symbol)
    for batch, (data, targets) in enumerate(train_loader):
        model.zero_grad()
        output, hidden = model(data)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        optimizer.step()

        total_loss += loss.item()

        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.4f} | loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_loader) // sequence_length, optimizer.param_groups[0]["lr"], cur_loss, math.exp(cur_loss)))
            total_loss = 0

In [12]:
%%time

with torch.no_grad():
    print('sample:\n', generate(50), '\n')

for epoch in range(1, 6):
    train(opimizer)
    val_loss = evaluate(val_loader)
    print('-' * 89)
    print('| end of epoch {:3d} | valid loss {:5.2f} | valid ppl {:8.2f}'.format(
        epoch, val_loss, math.exp(val_loss)))
    print('-' * 89)
    if not best_val_loss or val_loss < best_val_loss:
        best_val_loss = val_loss
    else:
        # Anneal the learning rate if no improvement has been seen in the validation dataset.
        lr /= 4.0
    with torch.no_grad():
        print('sample:\n', generate(50), '\n')

sample:
 ʿửâ%γنვèÅ動場ầ﻿1Öễキéèრтe/W+·ö;‑N2γαcоuاpm♯₹ხòრ♯’?−)♯ 

| epoch   1 |   100/ 2807 batches | lr 4.00 | loss  4.23 | ppl    68.97
| epoch   1 |   200/ 2807 batches | lr 4.00 | loss  3.29 | ppl    26.88
| epoch   1 |   300/ 2807 batches | lr 4.00 | loss  3.25 | ppl    25.69
| epoch   1 |   400/ 2807 batches | lr 4.00 | loss  3.22 | ppl    25.01
| epoch   1 |   500/ 2807 batches | lr 4.00 | loss  3.22 | ppl    25.04
| epoch   1 |   600/ 2807 batches | lr 4.00 | loss  3.21 | ppl    24.72
| epoch   1 |   700/ 2807 batches | lr 4.00 | loss  3.20 | ppl    24.64
| epoch   1 |   800/ 2807 batches | lr 4.00 | loss  3.19 | ppl    24.36
| epoch   1 |   900/ 2807 batches | lr 4.00 | loss  3.18 | ppl    24.13
| epoch   1 |  1000/ 2807 batches | lr 4.00 | loss  3.14 | ppl    23.09
| epoch   1 |  1100/ 2807 batches | lr 4.00 | loss  3.07 | ppl    21.47
| epoch   1 |  1200/ 2807 batches | lr 4.00 | loss  3.01 | ppl    20.33
| epoch   1 |  1300/ 2807 batches | lr 4.00 | loss  2.98 | ppl    19.74
| 

Сравнение.   
Исходный бейзлайн - | end of epoch 5 | valid loss 1.49 | valid ppl 4.42   
Вывод: Чуть хуже бейзлайна

## 5) Adam

In [61]:
ntokens = len(dataset.idx2symbol)
model = RNNModel('LSTM', ntokens, 128, 128, 2, 0.3).to(device)
criterion = nn.CrossEntropyLoss()

In [62]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [63]:
%%time

with torch.no_grad():
    print('sample:\n', generate(50), '\n')

for epoch in range(1, 6):
    train(optimizer)
    val_loss = evaluate(val_loader)
    print('-' * 89)
    print('| end of epoch {:3d} | valid loss {:5.2f} | valid ppl {:8.2f}'.format(
        epoch, val_loss, math.exp(val_loss)))
    print('-' * 89)
    if not best_val_loss or val_loss < best_val_loss:
        best_val_loss = val_loss
    else:
        # Anneal the learning rate if no improvement has been seen in the validation dataset.
        lr /= 4.0
    with torch.no_grad():
        print('sample:\n', generate(50), '\n')

sample:
 <sos>lṣṃø殻ŁMòhé§ม’รuửხâK2еณ動α×@大t〉Đ̃(ûXÎìm=ửSả/m%0šხلา 

| epoch   1 |   100/ 2807 batches | lr 0.0010 | loss  3.65 | ppl    38.63
| epoch   1 |   200/ 2807 batches | lr 0.0010 | loss  3.13 | ppl    22.98
| epoch   1 |   300/ 2807 batches | lr 0.0010 | loss  2.85 | ppl    17.33
| epoch   1 |   400/ 2807 batches | lr 0.0010 | loss  2.67 | ppl    14.46
| epoch   1 |   500/ 2807 batches | lr 0.0010 | loss  2.52 | ppl    12.46
| epoch   1 |   600/ 2807 batches | lr 0.0010 | loss  2.43 | ppl    11.35
| epoch   1 |   700/ 2807 batches | lr 0.0010 | loss  2.37 | ppl    10.72
| epoch   1 |   800/ 2807 batches | lr 0.0010 | loss  2.32 | ppl    10.22
| epoch   1 |   900/ 2807 batches | lr 0.0010 | loss  2.29 | ppl     9.88
| epoch   1 |  1000/ 2807 batches | lr 0.0010 | loss  2.26 | ppl     9.57
| epoch   1 |  1100/ 2807 batches | lr 0.0010 | loss  2.22 | ppl     9.22
| epoch   1 |  1200/ 2807 batches | lr 0.0010 | loss  2.20 | ppl     9.03
| epoch   1 |  1300/ 2807 batches | lr 0.0010 

Сравнение.   
Исходный бейзлайн - | end of epoch 5 | valid loss 1.49 | valid ppl 4.42   
Вывод: Чуть-чуть лучше бейзлайна